# 用于对本月的历史数据按照板块名称进行分组统计，结果保存到result.csv文件

In [27]:
import pandas as pd

In [28]:
def get_data() -> tuple[pd.DataFrame, list]:
    """
    获得股票历史信息，并计算总市值
    """
    # 显示结果
    df = pd.read_csv(
        "../data/Hist_2023-03-23.csv", parse_dates=['日期'], index_col=0, dtype={"股票代码": object})
    # dates = df.index.unique().sort_values().to_list()
    # print(type(dates[0]))
    # dates = [x.strftime("%Y-%m-%d") for x in dates]
    # 获得当前结果集的日期列表
    # dates_list = [date.strftime('%Y-%m-%d') for date in dates]
    value = pd.read_csv("../data/总股本.csv", index_col=0, dtype={"代码": object})
    value_dict = value['总股本'].to_dict()
    df['总股本'] = df['股票代码'].apply(lambda x: value_dict.get(x))
    df['总市值'] = df['总股本']*df['收盘']

    return df


In [29]:
df = get_data()
df.info()


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 81327 entries, 2023-03-01 to 2023-03-22
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   开盘      81327 non-null  float64
 1   收盘      81327 non-null  float64
 2   最高      81327 non-null  float64
 3   最低      81327 non-null  float64
 4   成交量     81327 non-null  int64  
 5   成交额     81327 non-null  float64
 6   振幅      81327 non-null  float64
 7   涨跌幅     81327 non-null  float64
 8   涨跌额     81327 non-null  float64
 9   换手率     81327 non-null  float64
 10  板块名称    81327 non-null  object 
 11  股票代码    81327 non-null  object 
 12  总股本     77740 non-null  float64
 13  总市值     77740 non-null  float64
dtypes: float64(11), int64(1), object(2)
memory usage: 9.3+ MB


In [6]:
df.columns

Index(['开盘', '收盘', '最高', '最低', '成交量', '成交额', '振幅', '涨跌幅', '涨跌额', '换手率', '板块名称',
       '股票代码', '总股本', '总市值'],
      dtype='object')

## 按日期、板块名称分组，并统计涨幅大于0和小于0的股票数量

In [30]:
# cur_date="2023-03-01"
# cur_df = df.loc[cur_date]
cur_df = df.copy()
# 按股票名称分组，并统计涨幅大于0和小于0的股票数量
result = cur_df.groupby(['日期', '板块名称'])['涨跌幅'].agg(
    [('涨的数量', lambda x: sum(x > 0)), ('跌的数量', lambda x: sum(x < 0))])
result['涨幅比'] = result['涨的数量']/(result['涨的数量']+result['跌的数量'])*100
result


涨的数量  跌的数量        涨幅比
日期         板块名称                           
2023-03-01 IT服务       120     7  94.488189
           LED         30     7  81.081081
           专业工程        31     3  91.176471
           专业服务        20     6  76.923077
           专业连锁         6     1  85.714286
...                   ...   ...        ...
2023-03-22 食品及饲料添加剂    12     6  66.666667
           餐饮           1     2  33.333333
           饰品           5    11  31.250000
           饲料          10     6  62.500000
           高速公路        11     4  73.333333

[3760 rows x 3 columns]

## 按日期、板块名称分组，并统计涨幅平均值以及总市值求和

In [31]:
cur_df = df.copy()
value_df = cur_df.groupby(['日期', "板块名称"]).agg(
    {"涨跌幅": "mean", "总市值": "sum"})
value_df


涨跌幅           总市值
日期         板块名称                            
2023-03-01 IT服务      4.119685  1.235165e+12
           LED       1.122821  2.941001e+11
           专业工程      1.343529  3.725464e+11
           专业服务      0.585556  1.848354e+11
           专业连锁      0.792500  6.418736e+10
...                       ...           ...
2023-03-22 食品及饲料添加剂  0.381500  1.600490e+11
           餐饮       -1.056667  2.176248e+10
           饰品       -0.651875  1.228474e+11
           饲料       -0.125000  2.814000e+11
           高速公路     -0.045000  2.341685e+11

[3760 rows x 2 columns]

## 合并结果

In [32]:
# cur_df = cur_df[(cur_df['总市值'] >= (start_value)*100_000_000)
#                 & (cur_df['总市值'] <= (end_value)*100_000_000)]
# cur_df

final_df = result.join(value_df, on=["日期","板块名称"])
# final_df.dropna(inplace=True, axis=0)
# 将Salary列格式化为亿元
final_df['总市值亿元'] = final_df['总市值'].apply(
    lambda x: '{:.2f}'.format(x/100000000))
final_df


涨的数量  跌的数量        涨幅比       涨跌幅           总市值     总市值亿元
日期         板块名称                                                             
2023-03-01 IT服务       120     7  94.488189  4.119685  1.235165e+12  12351.65
           LED         30     7  81.081081  1.122821  2.941001e+11   2941.00
           专业工程        31     3  91.176471  1.343529  3.725464e+11   3725.46
           专业服务        20     6  76.923077  0.585556  1.848354e+11   1848.35
           专业连锁         6     1  85.714286  0.792500  6.418736e+10    641.87
...                   ...   ...        ...       ...           ...       ...
2023-03-22 食品及饲料添加剂    12     6  66.666667  0.381500  1.600490e+11   1600.49
           餐饮           1     2  33.333333 -1.056667  2.176248e+10    217.62
           饰品           5    11  31.250000 -0.651875  1.228474e+11   1228.47
           饲料          10     6  62.500000 -0.125000  2.814000e+11   2814.00
           高速公路        11     4  73.333333 -0.045000  2.341685e+11   2341.69

[3760 rows x 6 columns]

## 按照涨跌幅的区间进行统计

In [33]:
# 按涨跌幅统计
x = ["跌停", "跌<-5%",  "-3%>-5%",     "-1-3%",    "平盘",
     "<3%",     "3-5%",   "5%-涨停", "涨停"]

db = df.copy()
# db = df.loc['2023-03-01']
bins = [-20, -10, -5, -3, -0.099, 0.099, 3, 5, 10, 20]
# bins = list(range(-11, 12))
cuts = pd.cut(db['涨跌幅'], bins=bins)

x = ["跌停", "跌<-5%",  "-5<-3%",     "-1<3%",    "平盘",
     "1<3%",     "3-5%",   "5%-涨停", "涨停"]
cuts.categories =x
cuts

日期
2023-03-01      (0.099, 3.0]
2023-03-02      (-5.0, -3.0]
2023-03-03    (-3.0, -0.099]
2023-03-06      (0.099, 3.0]
2023-03-07    (-3.0, -0.099]
                   ...      
2023-03-16      (0.099, 3.0]
2023-03-17    (-3.0, -0.099]
2023-03-20    (-3.0, -0.099]
2023-03-21      (0.099, 3.0]
2023-03-22    (-3.0, -0.099]
Name: 涨跌幅, Length: 81327, dtype: category
Categories (9, interval[float64, right]): [(-20.0, -10.0] < (-10.0, -5.0] < (-5.0, -3.0] < (-3.0, -0.099] ... (0.099, 3.0] < (3.0, 5.0] < (5.0, 10.0] < (10.0, 20.0]]

In [34]:
pct_chg_list = db.groupby(["日期","板块名称", cuts])['涨跌幅'].count()
pct_chg_list


日期          板块名称  涨跌幅            
2023-03-01  IT服务  (-20.0, -10.0]      0
                  (-10.0, -5.0]       0
                  (-5.0, -3.0]        0
                  (-3.0, -0.099]      6
                  (-0.099, 0.099]     1
                                     ..
2023-03-22  高速公路  (-0.099, 0.099]     5
                  (0.099, 3.0]       11
                  (3.0, 5.0]          0
                  (5.0, 10.0]         0
                  (10.0, 20.0]        0
Name: 涨跌幅, Length: 33840, dtype: int64

In [13]:
pct_chg_list.index



MultiIndex([('2023-03-01', 'IT服务',  (-20.0, -10.0]),
            ('2023-03-01', 'IT服务',   (-10.0, -5.0]),
            ('2023-03-01', 'IT服务',    (-5.0, -3.0]),
            ('2023-03-01', 'IT服务',  (-3.0, -0.099]),
            ('2023-03-01', 'IT服务', (-0.099, 0.099]),
            ('2023-03-01', 'IT服务',    (0.099, 3.0]),
            ('2023-03-01', 'IT服务',      (3.0, 5.0]),
            ('2023-03-01', 'IT服务',     (5.0, 10.0]),
            ('2023-03-01', 'IT服务',    (10.0, 20.0]),
            ('2023-03-01',  'LED',  (-20.0, -10.0]),
            ...
            ('2023-03-22',   '饲料',    (10.0, 20.0]),
            ('2023-03-22', '高速公路',  (-20.0, -10.0]),
            ('2023-03-22', '高速公路',   (-10.0, -5.0]),
            ('2023-03-22', '高速公路',    (-5.0, -3.0]),
            ('2023-03-22', '高速公路',  (-3.0, -0.099]),
            ('2023-03-22', '高速公路', (-0.099, 0.099]),
            ('2023-03-22', '高速公路',    (0.099, 3.0]),
            ('2023-03-22', '高速公路',      (3.0, 5.0]),
            ('2023-03-22', '高速

In [36]:
# db = pd.DataFrame(pct_chg_list)
# db.reset_index(inplace=True,level=[0, 1])
db = pct_chg_list.unstack()
db


涨跌幅                  (-20.0, -10.0]  (-10.0, -5.0]  (-5.0, -3.0]  \
日期         板块名称                                                    
2023-03-01 IT服务                   0              0             0   
           LED                    0              0             0   
           专业工程                   0              0             0   
           专业服务                   0              0             0   
           专业连锁                   0              1             0   
...                             ...            ...           ...   
2023-03-22 食品及饲料添加剂               0              0             0   
           餐饮                     0              0             0   
           饰品                     0              1             0   
           饲料                     0              1             0   
           高速公路                   0              0             1   

涨跌幅                  (-3.0, -0.099]  (-0.099, 0.099]  (0.099, 3.0]  \
日期         板块名称                                                      
2023-03-01 IT服务                   6                1            41   
           LED                    5                5            27   
           专业工程                   3                1            28   
           专业服务                   6                4            14   
           专业连锁                   0                1             5   
...                             ...              ...           ...   
2023-03-22 食品及饲料添加剂               4                4            12   
           餐饮                     2                0             1   
           饰品                    10                0             5   
           饲料                     5                3             9   
           高速公路                   3                5            11   

涨跌幅                  (3.0, 5.0]  (5.0, 10.0]  (10.0, 20.0]  
日期         板块名称                                             
2023-03-01 IT服务              45           29             4  
           LED                1            1             0  
           专业工程               1            1             0  
           专业服务               3            0             0  
           专业连锁               0            1             0  
...                         ...          ...           ...  
2023-03-22 食品及饲料添加剂           0            0             0  
           餐饮                 0            0             0  
           饰品                 0            0             0  
           饲料                 0            0             0  
           高速公路               0            0             0  

[3760 rows x 9 columns]

In [37]:
result = pd.merge(final_df, db, on=["日期",'板块名称'])
result.head(10)


涨的数量  跌的数量        涨幅比       涨跌幅           总市值     总市值亿元  \
日期         板块名称                                                             
2023-03-01 IT服务    120     7  94.488189  4.119685  1.235165e+12  12351.65   
           LED      30     7  81.081081  1.122821  2.941001e+11   2941.00   
           专业工程     31     3  91.176471  1.343529  3.725464e+11   3725.46   
           专业服务     20     6  76.923077  0.585556  1.848354e+11   1848.35   
           专业连锁      6     1  85.714286  0.792500  6.418736e+10    641.87   
           个护用品      4     8  33.333333 -0.180769  6.866488e+10    686.65   
           中药       38    27  58.461538  0.256528  1.086376e+12  10863.76   
           乘用车       8     1  88.888889  1.413333  9.225317e+11   9225.32   
           乳品        9     9  50.000000  0.038333  3.096878e+11   3096.88   
           互联网电商    14     2  87.500000  1.166667  1.367550e+11   1367.55   

                  (-20.0, -10.0]  (-10.0, -5.0]  (-5.0, -3.0]  (-3.0, -0.099]  \
日期         板块名称                                                                 
2023-03-01 IT服务                0              0             0               6   
           LED                 0              0             0               5   
           专业工程                0              0             0               3   
           专业服务                0              0             0               6   
           专业连锁                0              1             0               0   
           个护用品                0              0             0               8   
           中药                  0              0             1              26   
           乘用车                 0              0             0               1   
           乳品                  0              0             1               7   
           互联网电商               0              0             0               2   

                  (-0.099, 0.099]  (0.099, 3.0]  (3.0, 5.0]  (5.0, 10.0]  \
日期         板块名称                                                            
2023-03-01 IT服务                 1            41          45           29   
           LED                  5            27           1            1   
           专业工程                 1            28           1            1   
           专业服务                 4            14           3            0   
           专业连锁                 1             5           0            1   
           个护用品                 1             4           0            0   
           中药                   8            35           1            1   
           乘用车                  0             7           1            0   
           乳品                   2             8           0            0   
           互联网电商                2            14           0            0   

                  (10.0, 20.0]  
日期         板块名称                 
2023-03-01 IT服务              4  
           LED               0  
           专业工程              0  
           专业服务              0  
           专业连锁              0  
           个护用品              0  
           中药                0  
           乘用车               0  
           乳品                0  
           互联网电商             0

In [38]:
result.tail()

涨的数量  跌的数量        涨幅比       涨跌幅           总市值    总市值亿元  \
日期         板块名称                                                               
2023-03-22 食品及饲料添加剂    12     6  66.666667  0.381500  1.600490e+11  1600.49   
           餐饮           1     2  33.333333 -1.056667  2.176248e+10   217.62   
           饰品           5    11  31.250000 -0.651875  1.228474e+11  1228.47   
           饲料          10     6  62.500000 -0.125000  2.814000e+11  2814.00   
           高速公路        11     4  73.333333 -0.045000  2.341685e+11  2341.69   

                     (-20.0, -10.0]  (-10.0, -5.0]  (-5.0, -3.0]  \
日期         板块名称                                                    
2023-03-22 食品及饲料添加剂               0              0             0   
           餐饮                     0              0             0   
           饰品                     0              1             0   
           饲料                     0              1             0   
           高速公路                   0              0             1   

                     (-3.0, -0.099]  (-0.099, 0.099]  (0.099, 3.0]  \
日期         板块名称                                                      
2023-03-22 食品及饲料添加剂               4                4            12   
           餐饮                     2                0             1   
           饰品                    10                0             5   
           饲料                     5                3             9   
           高速公路                   3                5            11   

                     (3.0, 5.0]  (5.0, 10.0]  (10.0, 20.0]  
日期         板块名称                                             
2023-03-22 食品及饲料添加剂           0            0             0  
           餐饮                 0            0             0  
           饰品                 0            0             0  
           饲料                 0            0             0  
           高速公路               0            0             0

In [39]:
result.describe()

,涨的数量,跌的数量,涨幅比,涨跌幅,总市值,"(-20.0, -10.0]","(-10.0, -5.0]","(-5.0, -3.0]","(-3.0, -0.099]","(-0.099, 0.099]","(0.099, 3.0]","(3.0, 5.0]","(5.0, 10.0]","(10.0, 20.0]"
count,3760.000000,3760.000000,3758.000000,3760.000000,3.760000e+03,3760.000000,3760.000000,3760.000000,3760.000000,3760.000000,3760.000000,3760.000000,3760.000000,3760.000000
mean,9.743883,11.235372,46.495814,-0.059964,3.448104e+11,0.015957,0.238564,1.193085,9.581383,1.073670,7.966755,0.968351,0.482979,0.100532
std,14.140295,15.638241,32.446674,1.498843,5.365581e+11,0.129503,0.843265,3.188826,12.878968,1.743654,11.216637,2.540611,1.553593,0.433377
min,0.000000,0.000000,0.000000,-5.855000,1.425863e+09,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.000000,3.000000,16.666667,-1.042029,7.872162e+10,0.000000,0.000000,0.000000,2.000000,0.000000,2.000000,0.000000,0.000000,0.000000
50%,5.000000,7.000000,45.553360,-0.041833,1.873977e+11,0.000000,0.000000,0.000000,6.000000,0.000000,4.000000,0.000000,0.000000,0.000000
75%,12.000000,14.000000,75.000000,0.754650,3.636402e+11,0.000000,0.000000,1.000000,12.000000,1.000000,10.000000,1.000000,0.000000,0.000000
max,196.000000,201.000000,100.000000,19.390000,4.559494e+12,2.000000,15.000000,55.000000,161.000000,25.000000,150.000000,45.000000,29.000000,7.000000


## 重命名列

In [42]:
result.info()


<class 'pandas.core.frame.DataFrame'>
MultiIndex: 3760 entries, (Timestamp('2023-03-01 00:00:00'), 'IT服务') to (Timestamp('2023-03-22 00:00:00'), '高速公路')
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   涨的数量             3760 non-null   int64  
 1   跌的数量             3760 non-null   int64  
 2   涨幅比              3758 non-null   float64
 3   涨跌幅              3760 non-null   float64
 4   总市值              3760 non-null   float64
 5   总市值亿元            3760 non-null   object 
 6   (-20.0, -10.0]   3760 non-null   int64  
 7   (-10.0, -5.0]    3760 non-null   int64  
 8   (-5.0, -3.0]     3760 non-null   int64  
 9   (-3.0, -0.099]   3760 non-null   int64  
 10  (-0.099, 0.099]  3760 non-null   int64  
 11  (0.099, 3.0]     3760 non-null   int64  
 12  (3.0, 5.0]       3760 non-null   int64  
 13  (5.0, 10.0]      3760 non-null   int64  
 14  (10.0, 20.0]     3760 non-null   int64  
dtypes: float64(3), int64(11), o

In [43]:
result.reset_index(inplace=True,level=[0,1])
result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3760 entries, 0 to 3759
Data columns (total 17 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   日期               3760 non-null   datetime64[ns]
 1   板块名称             3760 non-null   object        
 2   涨的数量             3760 non-null   int64         
 3   跌的数量             3760 non-null   int64         
 4   涨幅比              3758 non-null   float64       
 5   涨跌幅              3760 non-null   float64       
 6   总市值              3760 non-null   float64       
 7   总市值亿元            3760 non-null   object        
 8   (-20.0, -10.0]   3760 non-null   int64         
 9   (-10.0, -5.0]    3760 non-null   int64         
 10  (-5.0, -3.0]     3760 non-null   int64         
 11  (-3.0, -0.099]   3760 non-null   int64         
 12  (-0.099, 0.099]  3760 non-null   int64         
 13  (0.099, 3.0]     3760 non-null   int64         
 14  (3.0, 5.0]       3760 non-null   int64  

In [45]:
x = ["跌停", "跌<-5%",  "-5<-3%",     "-1<3%",    "平盘",
     "1<3%",     "3-5%",   "5%-涨停", "涨停"]
a = result.columns[:8].to_list()
a.extend(x)
# a
result.columns =a
result.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3760 entries, 0 to 3759
Data columns (total 17 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   日期      3760 non-null   datetime64[ns]
 1   板块名称    3760 non-null   object        
 2   涨的数量    3760 non-null   int64         
 3   跌的数量    3760 non-null   int64         
 4   涨幅比     3758 non-null   float64       
 5   涨跌幅     3760 non-null   float64       
 6   总市值     3760 non-null   float64       
 7   总市值亿元   3760 non-null   object        
 8   跌停      3760 non-null   int64         
 9   跌<-5%   3760 non-null   int64         
 10  -5<-3%  3760 non-null   int64         
 11  -1<3%   3760 non-null   int64         
 12  平盘      3760 non-null   int64         
 13  1<3%    3760 non-null   int64         
 14  3-5%    3760 non-null   int64         
 15  5%-涨停   3760 non-null   int64         
 16  涨停      3760 non-null   int64         
dtypes: datetime64[ns](1), float64(3), int64(11), object(

In [46]:
result.head(10)


,日期,板块名称,涨的数量,跌的数量,涨幅比,涨跌幅,总市值,总市值亿元,跌停,跌<-5%,-5<-3%,-1<3%,平盘,1<3%,3-5%,5%-涨停,涨停
0,2023-03-01,IT服务,120,7,94.488189,4.119685,1.235165e+12,12351.65,0,0,0,6,1,41,45,29,4
1,2023-03-01,LED,30,7,81.081081,1.122821,2.941001e+11,2941.00,0,0,0,5,5,27,1,1,0
2,2023-03-01,专业工程,31,3,91.176471,1.343529,3.725464e+11,3725.46,0,0,0,3,1,28,1,1,0
3,2023-03-01,专业服务,20,6,76.923077,0.585556,1.848354e+11,1848.35,0,0,0,6,4,14,3,0,0
4,2023-03-01,专业连锁,6,1,85.714286,0.792500,6.418736e+10,641.87,0,1,0,0,1,5,0,1,0
5,2023-03-01,个护用品,4,8,33.333333,-0.180769,6.866488e+10,686.65,0,0,0,8,1,4,0,0,0
6,2023-03-01,中药,38,27,58.461538,0.256528,1.086376e+12,10863.76,0,0,1,26,8,35,1,1,0
7,2023-03-01,乘用车,8,1,88.888889,1.413333,9.225317e+11,9225.32,0,0,0,1,0,7,1,0,0
8,2023-03-01,乳品,9,9,50.000000,0.038333,3.096878e+11,3096.88,0,0,1,7,2,8,0,0,0
9,2023-03-01,互联网电商,14,2,87.500000,1.166667,1.367550e+11,1367.55,0,0,0,2,2,14,0,0,0


In [47]:
result.to_csv("result_20230323.csv",index=False)

## 校验一下数据结果

In [52]:
df=pd.read_csv("result_20230323.csv",index_col=0,parse_dates=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3760 entries, 2023-03-01 to 2023-03-22
Data columns (total 16 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   板块名称    3760 non-null   object 
 1   涨的数量    3760 non-null   int64  
 2   跌的数量    3760 non-null   int64  
 3   涨幅比     3758 non-null   float64
 4   涨跌幅     3760 non-null   float64
 5   总市值     3760 non-null   float64
 6   总市值亿元   3760 non-null   float64
 7   跌停      3760 non-null   int64  
 8   跌<-5%   3760 non-null   int64  
 9   -5<-3%  3760 non-null   int64  
 10  -1<3%   3760 non-null   int64  
 11  平盘      3760 non-null   int64  
 12  1<3%    3760 non-null   int64  
 13  3-5%    3760 non-null   int64  
 14  5%-涨停   3760 non-null   int64  
 15  涨停      3760 non-null   int64  
dtypes: float64(4), int64(11), object(1)
memory usage: 499.4+ KB


In [53]:
df.head()

,板块名称,涨的数量,跌的数量,涨幅比,涨跌幅,总市值,总市值亿元,跌停,跌<-5%,-5<-3%,-1<3%,平盘,1<3%,3-5%,5%-涨停,涨停
日期,,,,,,,,,,,,,,,,
2023-03-01,IT服务,120,7,94.488189,4.119685,1.235165e+12,12351.65,0,0,0,6,1,41,45,29,4
2023-03-01,LED,30,7,81.081081,1.122821,2.941001e+11,2941.00,0,0,0,5,5,27,1,1,0
2023-03-01,专业工程,31,3,91.176471,1.343529,3.725464e+11,3725.46,0,0,0,3,1,28,1,1,0
2023-03-01,专业服务,20,6,76.923077,0.585556,1.848354e+11,1848.35,0,0,0,6,4,14,3,0,0
2023-03-01,专业连锁,6,1,85.714286,0.792500,6.418736e+10,641.87,0,1,0,0,1,5,0,1,0


In [ ]:
color = ["green", "green", "green", "green",
         "yellow", "red", "red", "red", "red"]


In [70]:
my_df=df.loc["2023-03-01"]
my_df=my_df[my_df['板块名称'] == "IT服务"]
x = ["跌停", "跌<-5%",  "-5<-3%",     "-1<3%",    "平盘",
     "1<3%",     "3-5%",   "5%-涨停", "涨停"]
color = ["green", "green", "green", "green",
         "yellow", "red", "red", "red", "red"]

my=my_df[x].unstack()
my=my.reset_index()
my.columns=['x',"date","y"]
# my
data = pd.DataFrame({"x": my["x"], "y": my["y"], "color": color})
data

,x,y,color
0,跌停,0,green
1,跌<-5%,0,green
2,-5<-3%,0,green
3,-1<3%,6,green
4,平盘,1,yellow
5,1<3%,41,red
6,3-5%,45,red
7,5%-涨停,29,red
8,涨停,4,red


In [51]:
df.groupby(df.index).sum() 

/var/folders/my/x1pz96mj4_l31xr74gb254fh0000gn/T/ipykernel_2272/600881992.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df.groupby(df.index).sum()


,涨的数量,跌的数量,涨幅比,涨跌幅,总市值,总市值亿元,跌停,跌<-5%,-5<-3%,-1<3%,平盘,1<3%,3-5%,5%-涨停,涨停
日期,,,,,,,,,,,,,,,
2023-03-01,3467,1387,16685.693392,173.281495,8.253078e+13,825307.82,0,15,66,1237,360,2838,361,162,33
2023-03-02,1704,3170,9230.437489,-51.697598,8.229636e+13,822963.55,3,72,218,2819,306,1433,127,79,19
2023-03-03,2214,2641,11603.483647,47.360306,8.256120e+13,825611.95,2,32,140,2382,386,1868,161,92,13
2023-03-06,2306,2579,10796.182765,27.092101,8.250207e+13,825020.73,2,27,155,2333,316,1825,267,139,15
2023-03-07,443,4587,2491.463954,-422.875794,8.122159e+13,812215.85,2,148,1167,3257,83,322,53,34,13
2023-03-08,3556,1338,16166.980986,153.886954,8.133020e+13,813301.97,0,8,64,1217,297,3050,293,120,29
2023-03-09,2347,2537,9720.758533,12.802962,8.123791e+13,812379.08,0,22,107,2347,318,1953,213,90,27
2023-03-10,741,4277,3051.409872,-318.969102,8.019045e+13,801904.46,6,101,436,3691,145,561,76,43,23
2023-03-13,2333,2571,11780.848361,48.692218,8.090180e+13,809018.03,1,33,162,2336,272,1851,251,147,32
